<a href="https://colab.research.google.com/github/rakeshcoolkat/Image_recognition-Python/blob/master/Hand_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyunpack
!pip install patool
import os
from pyunpack import Archive

local_zip = '/content/Assignment.rar'
Archive('/content/Assignment.rar').extractall('/content/sample_data')

  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack
     |████████████████████████████████| 81kB 4.9MB/s 


In [0]:
import tensorflow as tf
import cv2
import glob
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
import numpy
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Input,Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten

In [0]:
train_path='/content/sample_data/Assignment'
image_size=128
classes=os.listdir('/content/sample_data/Assignment')
def load_train(train_path, image_size, classes):
    images = []
    labels = []
    img_names = []

    print('Going to read training images')
    for fields in classes:   
        index = classes.index(fields)
        print('Now going to read {} files (Index: {})'.format(fields, index))
        path = os.path.join(train_path, fields, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            image = image.astype(np.float32)
            image = np.multiply(image, 1.0 / 255.0)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            #labels.append(fields)
            flbase = os.path.basename(fl)
            img_names.append(flbase)
    images = np.array(images)
    labels = np.array(labels)
    #labels=np.reshape(label,(-1,1))
    img_names = np.array(img_names)

    return images, labels, img_names

In [60]:

images, labels, img_names = load_train(train_path, image_size, classes)
images, labels, img_names = shuffle(images, labels, img_names) 
x_train,x_test,y_train,y_test=train_test_split(images,labels, test_size=0.33)
x_train = x_train.reshape(x_train.shape[0], 128, 128,3)
x_test = x_test.reshape(x_test.shape[0], 128, 128,3) 

Going to read training images
Now going to read LeftH files (Index: 0)
Now going to read rightH files (Index: 1)


In [0]:
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [62]:
model = Sequential()

model.add(Convolution2D(32,2,2,border_mode='same',input_shape=(128, 128,3)))
model.add(Activation('relu'))
model.add(Convolution2D(32,2,2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Convolution2D(64,3,3))
model.add(Activation('relu'))
model.add(Convolution2D(64,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Convolution2D(96,4,4))
model.add(Activation('relu'))
model.add(Convolution2D(96,4,4))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), input_shape=(128, 128,..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2))`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (4, 4))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: Use

In [0]:
 #model.compile(optimizer='adam',
 #               loss='mean_squared_error',
 #                 metrics=[r2_keras])

In [0]:
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
                metrics=['accuracy'])

In [64]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 128, 128, 32)      416       
_________________________________________________________________
activation_33 (Activation)   (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 127, 127, 32)      4128      
_________________________________________________________________
activation_34 (Activation)   (None, 127, 127, 32)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 61, 61, 64)        18496     
__________

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [130]:
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
  height_shift_range=0.2,
   horizontal_flip=False,vertical_flip=False)
datagen.fit(x_train)
hist=model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                         steps_per_epoch=len(x_train) / 32, 
                         epochs=10,
                         verbose = 2 ,
                         validation_data=(x_test,y_test))

Epoch 1/10
 - 29s - loss: 0.4001 - acc: 0.8112 - val_loss: 0.3081 - val_acc: 0.8864
Epoch 2/10
 - 28s - loss: 0.3989 - acc: 0.8147 - val_loss: 0.2617 - val_acc: 0.8939
Epoch 3/10
 - 28s - loss: 0.4063 - acc: 0.8287 - val_loss: 0.3002 - val_acc: 0.8939
Epoch 4/10
 - 28s - loss: 0.3678 - acc: 0.8497 - val_loss: 0.2945 - val_acc: 0.8561
Epoch 5/10
 - 28s - loss: 0.4196 - acc: 0.7816 - val_loss: 0.2362 - val_acc: 0.9242
Epoch 6/10
 - 28s - loss: 0.3555 - acc: 0.8619 - val_loss: 0.2406 - val_acc: 0.9167
Epoch 7/10
 - 28s - loss: 0.3347 - acc: 0.8636 - val_loss: 0.1838 - val_acc: 0.9318
Epoch 8/10
 - 28s - loss: 0.2974 - acc: 0.8933 - val_loss: 0.2811 - val_acc: 0.8561
Epoch 9/10
 - 28s - loss: 0.3550 - acc: 0.8654 - val_loss: 0.3325 - val_acc: 0.8333
Epoch 10/10
 - 28s - loss: 0.2603 - acc: 0.9318 - val_loss: 0.3136 - val_acc: 0.8485


In [131]:

model.evaluate(x_test, y_test)
model.save('my_model.h5')
y_pred=model.predict(x_test)
loss, acc = model.evaluate(x_test, y_test, verbose = 0)
print(acc * 100)



132/132 [==============================] - 3s 19ms/step
84.84848484848484


In [86]:
x_test.shape

(132, 128, 128, 3)

In [144]:
from google.colab.patches import cv2_imshow
images=[]
image = cv2.imread('/content/Hand_0000024.jpg')
image1 = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)

images.append(image1)
images = np.array(images, dtype=np.uint8)
images = images.astype('float32')
images = np.multiply(images, 1.0/255.0) 
x=model.predict(images)
#cv2_imshow(image)
print(x)
if(x[0,0]<x[0,1]):
  print("right hand")
else:
  print("left hand")



[[0.26112565 0.7388743 ]]
right hand
